<a href="https://colab.research.google.com/github/obin1/MLAQ/blob/master/MLAQ_photochem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Read in input data
data = pd.read_table('https://raw.githubusercontent.com/obin1/FormO3NOx/master/C_and_J.txt',sep=',')
data = data[data['Time [min]'] != 1440] # delete last row for each day
JC = data.iloc[:,2:14] #extract just J and C
S = pd.read_table('https://raw.githubusercontent.com/obin1/FormO3NOx/master/S.txt',sep=',')
S = S.iloc[:,2:13] # extract reaction rates
A = np.array([[0,  1, -1,  0,  0,  0,  0,  0,  0,  0],  # O3
     [1,  0, -1,  0,  0,  0, -1,  0,  0,  0],  # NO
     [-1,  0,  1,  0,  0,  0,  1, -1,  0,  0],  # NO2
     [0,  0,  0, -1, -1, -1,  0,  0,  0,  0],  # HCHO
     [0,  0,  0,  2,  0,  1, -1,  0,  0,  1], # HO2.
     [0,  0,  0,  0,  0,  0,  0,  0, -1, -1]])  # HO2H
#              0  0  0  0  0 -1  1 -1  2 -1;  # OH.
#              1 -1  0  0  0  0  0  0  0  0;  # O
#              0  0  0  0  0  0  0  1  0  0;  # HNO3
#              0  0  0  1  1  1  0  0  0  0;  # CO
#              0  0  0  0  1  0  0  0  0  0]; # H2

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
### Test/train split
JC_train, JC_test, S_train, S_test = train_test_split(JC, S, test_size=0.1)
JC_train = JC_train.to_numpy()
JC_test = JC_test.to_numpy()
S_train = S_train.to_numpy()
S_test = S_test.to_numpy()

In [ ]:
### Define Neural Net
class PhotoNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        """
        Two linear layers
        """
        super(PhotoNN, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.linear2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        """
        Foward pass with ReLU activation
        """
        # TODO: Implement this!
        h_relu = F.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return torch.squeeze(y_pred)


In [ ]:
def train(rxn):
    # Construct network according to # of species
    net = PhotoNN(len(JC_train), len(JC_train) + 6)  
    print(net)
    criterion = nn.MSELoss() # Use MSE Loss
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # Use SGD
    # load data
    #species = [0] # Get J
    #for s in range(len(A)): # Get relevant species
    #  if A[s,rxn] != 0:
    #    species.append(s+1)
    #species = [x + 1 for x in np.nonzero(A[:,rxn])] # Get relevant species
    train_data = JC_train
    train_labels = S_train[:,rxn]
    dev_data = JC_test
    dev_labels = S_test[:,rxn]
    # Training loop
    train_losses = []
    dev_losses = []
    running_loss = 0
    for i in range(len(train_data)):
        # run the model and backprop for train steps
        x = torch.from_numpy(train_data[i].astype(np.float32)).float()
        y = torch.tensor(train_labels[i],dtype=torch.float32)
        print(x)
        # Forward pass: Get preds
        y_pred = net(x)
        
        # Compute loss
        loss = criterion(y_pred, y)
        running_loss += (y_pred.tolist()-train_labels[i])**2
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # every 100 steps, log metrics
        if i % 1000 == 0:
          ypred = net(torch.from_numpy(dev_data.astype(np.float32)).float())
          devloss = sum((ypred.tolist()-dev_labels)**2)/len(ypred.tolist())
          train_losses.append(running_loss/i)
          dev_losses.append(devloss)
          #print('On step' + str(i) +': Train MSE '+ str(running_loss/i) +' | Dev MSE ' + str(devloss))

    plt.figure()
    plt.plot(train_losses)
    plt.plot(dev_losses)
    plt.title('R ' + str(rxn+1))
    plt.legend(['Train Losses', 'Dev Losses'])
    return net
    # save model
    #print('Done training. Saving model at ' + save)
    #torch.save(model, 'C:\Users\islam\OneDrive - Johns Hopkins\Wexler\Atmos\MLAQ\Pytorch\R' + str(rxn+1))

In [ ]:
net = []
for rxn in range(len(A.T)): # Train a net for each reaction
  net.append(train(rxn))


In [ ]:
# Gather test statistics
abs_err = []
rel_err = []
# for rxn in range(len(A.T)): 
#   species = [0] # Get J
#   for s in range(len(A)): # Get relevant species
#     if A[s,rxn] != 0:
#       species.append(s+1)
  dev_data = JC_test
  dev_labels = S_test[:,rxn]
  ypred = net[rxn](torch.from_numpy(dev_data.astype(np.float32)).float())
  abs_err.append(sum(abs(ypred.tolist()-dev_labels))/len(ypred.tolist()))
  rel_err.append(abs_err[rxn]/np.mean(dev_labels))
plt.figure()
plt.plot(range(1,11),abs_err)
plt.xlabel("reaction #")
plt.ylabel("Mean abs error")
plt.figure()
plt.plot(range(1,11),rel_err)
plt.xlabel("reaction #")
plt.ylabel("Mean rel. error (%)")
